In [1]:
#Caricare le librerie
import tweepy
import pandas as pd
import numpy as np
import re
import json

In [2]:
consumer_key= "rrR0Znz98oKkjfB8b72oLH9Qc"
consumer_secret= "m7iOPX6iW8R3IalRf7x56YVeIb18o3I1bhUftg8JzlQ6fAjLgW"
access_token= "1442250607-3cJYKYyh7Ae19nMcv5YoxOIB2XlvgVX0ROaDSJx"
access_token_secret= "ZOnaaCTgO7DsKtmPjVruOlkvzBY8p6MbkKLjiYLdamBYE"
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [9]:
#Scaricare in un dataframe i tweet in base ad un hashtag (until="2021-12-21")
#NB: hasattr(obj, name, /) Return whether the object has an attribute with the given name.
hashtag="#lockdown"
list_tweet=[]
list_originals=[]
list_retweetted=[]
n=4500
for tweet in tweepy.Cursor(api.search_tweets,q=hashtag,count=n,lang="it").items(n):

  #gestione dei retweet con try-except
  try:
    list_originals.append(tweet.retweeted_status.id) #creo lista dei retweets id 
    #lista tweets che sono stati retweettati
    list_retweetted.append([tweet.retweeted_status.created_at, tweet.retweeted_status.id, tweet.retweeted_status.text, 
                     tweet.retweeted_status.favorite_count, tweet.retweeted_status.retweet_count, 
                     tweet.retweeted_status.user.screen_name]) 
  except:
    list_originals.append(0)
  #creo lista e df dei tweet  
  list_tweet.append([tweet.created_at, tweet.id, tweet.text, 
                     tweet.favorite_count, tweet.retweet_count, 
                     tweet.user.location,tweet.user.screen_name,
                     hasattr(tweet, "retweeted_status"),
                     tweet.in_reply_to_screen_name])

                     
tweets = pd.DataFrame(list_tweet[1:], 
                      columns=['date','id','text','like','n_rt','location','author',
                               "retweet","reply_name"])
tweets["retwitted_id"]=list_originals[1:]
#creo df dei retweet
retweetted = pd.DataFrame(list_retweetted[1:], 
                      columns=['date','id','text','like','n_rt','author'])                                                    
tweets.head()

,date,id,text,like,n_rt,location,author,retweet,reply_name,retwitted_id
0,2021-12-23 14:33:10+00:00,1474025293430104076,RT @topolina12: Dopo due anni e più di #restri...,0,8,,FerranteViola,True,None,1473934055557189634
1,2021-12-23 14:31:27+00:00,1474024862498840583,RT @LiciaRonzulli: NELLA LEGGE DI BILANCIO MIS...,0,3,Italia,forza_italia,True,None,1474018619235442688
2,2021-12-23 14:23:45+00:00,1474022922549436424,@Nessuno01333091 @Medicoinfamigl1 @FortressSim...,0,0,,nd_ruug,False,Nessuno01333091,0
3,2021-12-23 14:12:34+00:00,1474020110264377350,RT @LiciaRonzulli: NELLA LEGGE DI BILANCIO MIS...,0,3,Bari,Frances69744637,True,None,1474018619235442688
4,2021-12-23 14:06:39+00:00,1474018619235442688,NELLA LEGGE DI BILANCIO MISURE E FONDI PER LA ...,2,3,Italia,LiciaRonzulli,False,None,0


In [10]:
#selezione dei tweet più retwettati 
retweetted.sort_values("n_rt",ascending=False).drop_duplicates()

,date,id,text,like,n_rt,author
853,2021-11-19 13:28:48+00:00,1461687908797227008,Mi sembra il giorno giusto per ripostare il fa...,2481,1332,darkskywriter
1149,2021-12-18 19:09:39+00:00,1472282934073872397,Che fate a Capodown?\n#lockdown,4759,669,lefrasidiosho
1727,2021-12-20 15:47:47+00:00,1472956905664520192,Noi invece Proponiamo il #Lockdown TOTALE e de...,851,638,EsercitoCrucian
1771,2021-12-18 22:56:11+00:00,1472339940327444482,Con il 90% di #vaccinati si moltiplicano i pos...,2237,595,intuslegens
802,2021-12-18 20:23:00+00:00,1472301392467476489,Saranno intitolate strade e piazze in suo onor...,1407,482,strange_days_82
...,...,...,...,...,...,...
1222,2021-12-20 22:03:35+00:00,1473051479216951296,Il 27 ho prenotato per la terza dose \nE non m...,49,1,LoopandLo
881,2021-12-21 18:42:44+00:00,1473363322913370114,Sento aria di #lockdown e voi? https://t.co/AP...,1,1,iFrancisPin
1219,2021-12-21 08:52:58+00:00,1473214904903647232,La mia libertà non finisce dove cominciano le ...,3,1,ChristianValse4
1216,2021-12-21 08:46:05+00:00,1473213171796221952,Questo è uno dei vergognosi sistemi di comunic...,2,1,ChristianValse4


##### Vado a vedere quali sono i tweet più retwittati e quante volte sono stati ritwittati dalla nostra rete (usando SQL)

In [8]:
#Open the terminal and run-   pip install U pandasql
#Open your Jupyter Notebook and in any cell run-    !pip install -U pandasql
import pandasql as ps
q1 = """SELECT id,like,n_rt, author,count() FROM retweetted group by id order by n_rt desc"""
print(ps.sqldf(q1, locals()))

                      id  like  n_rt           author  count()
0    1472956905664520192   851   637  EsercitoCrucian       98
1    1472339940327444482  2237   595      intuslegens        5
2    1472301392467476489  1407   482  strange_days_82        4
3    1473386206914359299  1015   339         fdragoni      251
4    1472289627205775372  1520   327       nonexpedit        2
..                   ...   ...   ...              ...      ...
119  1473340113560256518     9     1         Psyco012        1
120  1473337945625174022     6     1   alessand_mella        1
121  1473282970073059339     0     1     BabboleoNews        1
122  1473229107999776771     2     1    antondepierro        1
123  1472638825369382913     2     1    ContiMariapia        1

[124 rows x 5 columns]


#### Creazione input Grephi tramite scrittura file .csv

In [12]:
rt_tweets=tweets[tweets.retweet==True]

In [18]:
tweets_ids=rt_tweets[['id','author']]
retwitted_ids=retweetted[['id','author']]
rt_tweets_ids=pd.concat([tweets_ids,retwitted_ids]).drop_duplicates().reset_index(drop=True)

In [19]:
Nodes_df=pd.DataFrame({'Id':rt_tweets_ids.id,'Label':rt_tweets_ids.author,'Interval':None})

In [17]:
Edges_df=pd.DataFrame({'Source':rt_tweets.id,'Target':rt_tweets.retwitted_id,'Type':'Directed','Kind':None,'Id':range(1, len(rt_tweets)+1),'Label':None,'Interval':None,'Weight':1.0})

In [21]:
Nodes_df.to_csv('Nodes_df.csv', index=False)
Edges_df.to_csv('Edges_df.csv', index=False)